<a href="https://colab.research.google.com/github/951237/TIL/blob/master/TIL__200601_%EC%BD%94%EB%A1%9C%EB%82%98_%ED%98%84%ED%99%A9_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91_feat_%EC%9D%8C%EC%84%B1%EC%A7%80%EC%9B%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pyttsx3
!pip install SpeechRecognition

     |████████████████████████████████| 32.8MB 123kB/s 


In [0]:
import requests, json       # 
import pyttsx3      # 음성 글씨로 나타냄
import re, threading, time  # threading - 코드를 병렬로 실행시키기 위함.
import speech_recognition as sr     # 음성인식 라이브러리

In [0]:
# api키 값 - parsehub.com 사이트 
API_KEY = " "   
PROJECT_TOKEN = ""
RUN_TOKEN = ""

In [0]:
# 클래스 생성
class Data():
    # 클래스 초기화
    def __init__(self, api_key, project_token):
        self.api_key = api_key      # api키
        self.project_token = project_token  # 프로젝트 킥
        self.param = {
            "api_key" : self.api_key    # 파라미터 api키
        }
        self.data = self.get_data()     # 데이터 가져와서 데이터에 넣기. 함수가 아래에 정의 되어 있는데, 작동하네
    
    #  함수 - 데이터 가져오기
    def get_data(self):
        response = requests.get(f'https://www.parsehbu.com/api/v2/projects/{self.project_token}/last_ready_run/data', params=self.params)   # 리퀘스트로 내용가져오기
        data = json.loads(response.text)    # 가져온 json 내용을 데이터 변수에 지정
        return Data 

    # 함수 - 전체 확진자 데이터 얻기
    def get_total_cases(self):
        data = self.data['total']       # json파일에서 전체 값 가져오기

        for content in data:        # 반복문
            if content['name'] == 'Coronavirus Cases:':      # json에서 일치하는 값 가져오기
                return content['value']     # 값 리턴

    # 함수 - 전체 사망자 데이터 가져오기
    def get_total_deaths(self):     
        data = self.data['total']       # json에서 total 전체 가져오기

        for content in data:        # 반복문
            if content['name'] == 'Deaths:':    # 사망자 레이블 일치하는 경우
                return content['value']     # 데이터 반환 숫자 반환

        return "0"      # 해당없으면 0으로 반환

    # 함수 나라가져오기
    def get_country_data(self, country):
        data = self.data['country']     # 나라 데이터 호출

        for content in data:
            if content['name'].lower() == country.lower():      # 파라미터의 나라를 찾으면 값을 반환
                return content
        return "0"

    # 나라이름 리스트 만들기
    def get_list_of_countries(self):        
        countries = []      # 나라명을 담을 빈 리스트 생성

        for country in self.data['country']:        # 나라데이터에서
            countries.append(country['name'].lower())   #나라이름만 리스트에 담기

    # 데이터 업데이트 하기
    def update_data(self):  
        response = requests.post(f'https://www.parsehbu.com/api/v2/projects/{self.project_token}/run', params=self.params)      # 토큰 이용하여 

        def poll():
            time.sleep(0.1)     # 0.1초 쉬고
            old_data = self.data        # 기존데이터를 old데이터로 지정
            while True:     # 항상 반복문
                new_data = self.get_data()      # 가져온 데이터를 새 데이터로 지정하고
                if new_data != old_data:        # 만약에 예전 데이터랑 새로가져온 데이터가 다르면
                    self.data = new_data        # 새로운 데이터를 데이터로 지정
                    print('Data updated')       # 업데이트 문구 출력
                    break       # 반복문 빠져나가기
            time.sleep(5)       # 5초간 쉬자

        t = threading.Thread(target=poll)       # 타겟 지정시 함수의 '()'부분을 뺌. poll 함수를 병렬로 실행
        t.start()       # 병렬로 실행 시작

In [0]:
# 말한 텍스트를 글자로 출력
def speak(text):        
    engine = pyttsx3.init()     # tts함수 초기화. engin 오브젝트 생성
    engine.say(text)        # 텍스트를 문자로 출력 라이브러리에 담기
    engine.runAndWait()

# 음성인식 함수
def get_audio():       
    r = sr.Recognizer()     # 오브젝트 생성
    with sr.Microphone() as source:     # 마이크로 듣는데
        audio = r.listen(source)        # 들은 음성을 오디오로 지정
        said = ""       # 말한것은 초기화

    try:        
        said = r.recognize_google(audio)        # 오디오를 구글 인식함수로 처리해서 said에 담기

    except Exception as e:      # 혹시 모를 예외가 생기면
        print("Exception:", str(e))     # 오류 출력하기

    return said.lower()     # 들은 것들을 소문자로 바꾸어서 반환

In [0]:

# 메인함수
def main():
    print("Started Programs")       # 시작 문구 출력
    data = Data(API_KEY, PROJECT_TOKEN)     # 데이터 오브젝트 생성하기
    END_PHRASE = 'stop'     # 끝내기 변수 stop 저장
    country_list = data.get_list_of_countries()     # 나라이름 리스트로 저장

# 패턴인식
    TOTAL_PATTERNS = {      # 음성인식 받은 부분은 키값, 함수부분은 밸류값으로 딕셔너리로 이용함.
        re.compile("[\w\s] + total [\w\s] + cases") : data.get_total_cases,     # 신기한거 함수에 ()가 없음. \w\s 는 어떤 단어가 오더라도. 전체 확진자 묻고 담하기 패턴
        re.compile("[\w\s] + total cases") : data.get_total_cases,      # 전체 확진자 묻고 담하기 패턴
        re.compile("[\w\s] + total [\w\s] + deaths") : data.get_total_deaths,       # 전체 사망자 물었을 때 값 반환
        re.compile("[\w\s] + total deaths") : data.get_total_deaths     # 전체 사망자 물었을 때 값 반환
    }

    COUNTRY_PATTERNS = {
        re.compile("[\w\s] + cases [\w\s] + ") : lambda country : data.get_countries_data(country)['total_cases'],      # 해당 나라의 전체 확진자 반환
        re.compile("[\w\s] + deaths [\w\s] + ") : lambda country : data.get_countries_data(country)['total_deaths']     # 해당 나라의 전체 사망자 반환
    }

    UPDATE_COMMAND = "update"       # 업데이트 명령 지정

    while True:     # 계속 반복문
        print("Listening...")       # 시작문구 출력
        text = get_audio()      # 음성으로 받아서 텍스트에 저장
        print(text)     # 입력받은 텍스트 출력
        result = None       # result값은 초기화

        for pattern, func in COUNTRY_PATTERNS.items():      # 패턴 딕셔너리 호출 반복문
            if pattern.match(text):     # 입력받은 문구가 패턴이랑 일치하면
                words = set(text.split(" "))        # 텍스트를 공백으로 나눠서 word에 집합자료형으로 바꿈. 집합자료형은 중복을 허용하지 않음.
                for country in country_list:        # 나라 이름 리스트를 반복문으로 돌리면서
                    if country in words:        # word에 나라이름이 있으면
                        result = func(country)      # 패턴에 맞게 함수의 값을 출력(사망자 혹은 확진자)
                        break       # 나가자
        
        for pattern, func in TOTAL_PATTERNS.itmes():    # 전체 패턴 딕셔너리 호출 반복문
            if pattern.match(text):     # 입력받은 텍스트가 패턴에 일치하면
                result = func()     # 패턴에 해당하는 함수의 값을 반환
                break

    # 업데이트 하기
        if text == UPDATE_COMMAND:      # 무한반복문에서 업데이트 명령이 들어오면
            result = "Data is being updated. This may take a moment."   # 얻데이트 문구 출력
            data.update_data()      # 업데이트 하기

        if result:      # 결과값이 있으면
            speak(result)   # 결과를 음성으로 출력

        if text.find(END_PHRASE) != -1:     # 끝내는 명령어를 찾으면
            print("Exit")       # 문구 출력하고
            break       # 무한반복문 탈출하기
